**Soft margin support vector**


In [1]:
from google.colab import files
uploaded = files.upload()

Saving satimage.scale.t to satimage.scale.t
Saving satimage.scale to satimage.scale
Saving satimage.scale.tr to satimage.scale.tr
Saving satimage.scale.val to satimage.scale.val


***Question 1***

Step 1: Load and Preprocess the Dataset and Implement the SVM Classifier

In [2]:
import numpy as np
from sklearn.datasets import load_svmlight_file
from cvxopt import matrix, solvers

# Load the data
X_train, y_train = load_svmlight_file('satimage.scale.tr')
X_val, y_val = load_svmlight_file('satimage.scale.val')
X_test, y_test = load_svmlight_file('satimage.scale.t')

# Convert to dense format as cvxopt does not support sparse matrices
X_train = X_train.toarray()
X_val = X_val.toarray()
X_test = X_test.toarray()

def filter_classes(data, labels, class1, class2):
    filter_mask = (labels == class1) | (labels == class2)
    filtered_data = data[filter_mask]
    filtered_labels = np.where(labels[filter_mask] == class1, 1, -1)
    return filtered_data, filtered_labels

X_train_filtered, y_train_filtered = filter_classes(X_train, y_train, 4, 6)
X_test_filtered, y_test_filtered = filter_classes(X_test, y_test, 4, 6)

def formulate_qp_problem(data, labels, C=1.0):
    n_samples, n_features = data.shape
    P = matrix(np.outer(labels, labels) * np.dot(data, data.T))
    q = matrix(-np.ones((n_samples, 1)))
    G = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
    h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
    A = matrix(labels.reshape(1, -1).astype(np.double))
    b = matrix(np.zeros(1))
    return P, q, G, h, A, b


def construct_classifier(data, labels, alphas):
    support_vectors = data[alphas > 1e-4]
    support_labels = labels[alphas > 1e-4]
    alphas_sv = alphas[alphas > 1e-4]
    w = np.sum((alphas_sv * support_labels)[:, np.newaxis] * support_vectors, axis=0)
    b = np.mean(support_labels - np.dot(support_vectors, w))
    return w, b


def predict(data, w, b):
    return np.sign(np.dot(data, w) + b)

def calculate_accuracy(predictions, labels):
    return np.mean(predictions == labels)

P, q, G, h, A, b = formulate_qp_problem(X_train_filtered, y_train_filtered)
solvers.options['show_progress'] = False
solution = solvers.qp(P, q, G, h, A, b)
alphas = np.array(solution['x']).flatten()

w, b = construct_classifier(X_train_filtered, y_train_filtered, alphas)

test_predictions = predict(X_test_filtered, w, b)
accuracy = calculate_accuracy(test_predictions, y_test_filtered)
print("Accuracy on the test set:", accuracy)


Accuracy on the test set: 0.801762114537445


Step 2: Confusion matrix and evaluating the SVM Classifier


In [3]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def balanced_accuracy(conf_matrix):
    class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
    return np.mean(class_accuracies)

test_predictions = predict(X_test_filtered, w, b)

conf_matrix = confusion_matrix(y_test_filtered, test_predictions)

print("Confusion Matrix:\n", conf_matrix)

print("\nClassification Report:\n", classification_report(y_test_filtered, test_predictions, target_names=['Class 4', 'Class 6']))

bal_acc = balanced_accuracy(conf_matrix)
print("Balanced Accuracy:", bal_acc)


Confusion Matrix:
 [[400  70]
 [ 65 146]]

Classification Report:
               precision    recall  f1-score   support

     Class 4       0.86      0.85      0.86       470
     Class 6       0.68      0.69      0.68       211

    accuracy                           0.80       681
   macro avg       0.77      0.77      0.77       681
weighted avg       0.80      0.80      0.80       681

Balanced Accuracy: 0.7715034788746598


**Question 2**

Step 1: Import Libraries and Define RBF Kernel Function

In [4]:
import numpy as np
from sklearn.datasets import load_svmlight_file
from cvxopt import matrix, solvers
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report

X_train, y_train = load_svmlight_file('satimage.scale.tr')
X_val, y_val = load_svmlight_file('satimage.scale.val')
X_test, y_test = load_svmlight_file('satimage.scale.t')

def rbf_kernel(x1, x2, gamma):
    return np.exp(-gamma * np.linalg.norm(x1 - x2, axis=1) ** 2)

Step 2: Formulate QP Problem with RBF Kernel

In [5]:
def formulate_qp_rbf(data, labels, C, gamma):
    n_samples, n_features = data.shape
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        K[i, :] = rbf_kernel(data[i], data, gamma)
    P = matrix(np.outer(labels, labels) * K)
    q = matrix(-np.ones((n_samples, 1)))
    G = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
    h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
    A = matrix(labels.reshape(1, -1).astype(np.double))
    b = matrix(np.zeros(1))

    return P, q, G, h, A, b

Step 3: Cross-validation to find best C and gamma

In [6]:
kf = KFold(n_splits=5)
best_accuracy = 0
best_params = {'C': 1.0, 'gamma': 1.0}

for C in [0.1, 1, 10]:
    for gamma in [0.1, 1, 10]:
        cv_accuracy = []
        for train_index, val_index in kf.split(X_train_filtered):
            X_train_cv, X_val_cv = X_train_filtered[train_index], X_train_filtered[val_index]
            y_train_cv, y_val_cv = y_train_filtered[train_index], y_train_filtered[val_index]

            P, q, G, h, A, b = formulate_qp_rbf(X_train_cv, y_train_cv, C, gamma)
            solvers.options['show_progress'] = False
            solution = solvers.qp(P, q, G, h, A, b)
            alphas = np.array(solution['x']).flatten()

            w, b = construct_classifier(X_train_cv, y_train_cv, alphas)
            val_predictions = predict(X_val_cv, w, b)
            cv_accuracy.append(calculate_accuracy(val_predictions, y_val_cv))

        avg_accuracy = np.mean(cv_accuracy)
        if avg_accuracy > best_accuracy:
            best_accuracy = avg_accuracy
            best_params = {'C': C, 'gamma': gamma}

Step 4: Train final model with best hyperparameters

In [7]:
P, q, G, h, A, b = formulate_qp_rbf(X_train_filtered, y_train_filtered, **best_params)
solution = solvers.qp(P, q, G, h, A, b)
alphas = np.array(solution['x']).flatten()
w, b = construct_classifier(X_train_filtered, y_train_filtered, alphas)

def predict(data, w, b):
    return np.sign(np.dot(data, w) + b)

def calculate_balanced_accuracy(conf_matrix):
    class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
    return np.mean(class_accuracies)

test_predictions = predict(X_test_filtered, w, b)

Step 5: display the result

In [8]:
conf_matrix = confusion_matrix(y_test_filtered, test_predictions)
print("Confusion Matrix:\n", conf_matrix)

accuracy = calculate_accuracy(test_predictions, y_test_filtered)
print("Accuracy on the test set:", accuracy)

bal_acc = calculate_balanced_accuracy(conf_matrix)
print("Balanced Accuracy:", bal_acc)

Confusion Matrix:
 [[361 109]
 [ 38 173]]
Accuracy on the test set: 0.7841409691629956
Balanced Accuracy: 0.7939951598265604


**Question 3**

Step 1: Load and Preprocess the Dataset

In [9]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.datasets import load_svmlight_file

X_train, y_train = load_svmlight_file('satimage.scale.tr')
X_val, y_val = load_svmlight_file('satimage.scale.val')
X_test, y_test = load_svmlight_file('satimage.scale.t')

X_train = X_train.toarray()
X_val = X_val.toarray()
X_test = X_test.toarray()

Step 2:Train a linear SVM classifier and Predict on the test set and calculate accuracy

In [10]:
svm_classifier = SVC(C=1, kernel='linear')

svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.8615


Step 3:Best hyperparameters on validation set and calculate the accuracy on the test set

In [11]:
C_values = [1, 10, 100]
gamma_values = [0.001, 0.01, 0.1, 1]

best_accuracy = 0
best_params = {'C': None, 'gamma': None}

for C in C_values:
    for gamma in gamma_values:
        svm_classifier = SVC(C=C, kernel='rbf', gamma=gamma)
        svm_classifier.fit(X_train, y_train)

        y_val_pred = svm_classifier.predict(X_val)
        accuracy = accuracy_score(y_val, y_val_pred)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'C': C, 'gamma': gamma}


final_clf = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel='rbf')
final_clf.fit(X_train, y_train)

test_predictions = final_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)


print(f"Best hyperparameters on validation set: {best_params}")
print("Accuracy on the test set:", test_accuracy)

Best hyperparameters on validation set: {'C': 10, 'gamma': 1}
Accuracy on the test set: 0.9045


Step 4:Calculate and print the confusion matrix and classification report and balanced accuracy

In [15]:
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score

conf_matrix = confusion_matrix(y_test, test_predictions)

class_report = classification_report(y_test, test_predictions, target_names=['Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6'])

balanced_acc = balanced_accuracy_score(y_test, test_predictions)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)
print(f"\nBalanced Accuracy: {balanced_acc:.4f}")


Confusion Matrix:
[[454   1   2   0   4   0]
 [  0 219   1   0   3   1]
 [  3   1 371  16   1   5]
 [  0   3  33 138   1  36]
 [  3   4   1   2 218   9]
 [  1   1  15  31  13 409]]

Classification Report:
              precision    recall  f1-score   support

     Class 1       0.98      0.98      0.98       461
     Class 2       0.96      0.98      0.97       224
     Class 3       0.88      0.93      0.90       397
     Class 4       0.74      0.65      0.69       211
     Class 5       0.91      0.92      0.91       237
     Class 6       0.89      0.87      0.88       470

    accuracy                           0.90      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.90      0.90      0.90      2000


Balanced Accuracy: 0.8902


<div dir="rtl" style="font-family: BNazanin; text-align: justify;">
در فرآیند انتخاب مقادیر خاص برای پارامترهای هایپرپارامتر مانند `C` و `gamma` در SVM با هسته RBF، ابتدا با یک دامنه گسترده از مقادیر شروع می‌کنیم تا درک کنیم که چگونه مدل به این پارامترها حساس است. سپس، برای هر ترکیب از `C` و `gamma`، یک مدل SVM روی داده‌های آموزشی آموزش داده می‌شود و عملکرد آن‌ها روی داده‌های اعتبارسنجی بررسی می‌شود.

اگر با افزایش `C` دقت بهبود یابد، ممکن است نشان دهنده کمبود تنظیم در مقادیر کمتر `C` باشد. همچنین، اگر با افزایش `gamma` دقت کاهش یابد، ممکن است نشان دهنده بیش‌تنظیمی مدل و حساسیت بیش از حد به نویز در داده‌ها باشد.

با توجه به نتایج اولیه، دامنه‌های `C` و `gamma را برای یافتن بهترین ترکیب تنظیم می‌کنیم. پس از یافتن بهترین پارامترها، مدل نهایی روی داده‌های تست برای ارزیابی عملکرد آن در مقابله با داده‌های دیده نشده ارزیابی می‌شود. این فرآیند کاملاً تجربی و تکراری است و بسیار بستگی به نتایج تجربی از ارزیابی‌های مدل روی داده‌های اعتبارسنجی دارد.
</div>







**Question 4**

In [17]:
from sklearn.datasets import load_svmlight_file
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

X_train, y_train = load_svmlight_file('satimage.scale.tr')
X_test, y_test = load_svmlight_file('satimage.scale.t')

X_train = X_train.toarray()
X_test = X_test.toarray()

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = SVC(kernel='rbf', C=1.0)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on the test data:", accuracy)


Accuracy on the test data: 0.889
